Install tensorflow package. 

Note: Turn on internet toggle from Settings panel located on the right

In [ ]:
!pip install tensorflow==2.0.0-alpha0

import necessary libraries

In [1]:
import numpy as np 
import pandas as pd 
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import matplotlib.pyplot as plt 
from sklearn.preprocessing import OneHotEncoder 

read in the data. this was written on the kaggle platform so we can use the dataset directly without downloading it

In [2]:
banknotes = pd.read_csv("/kaggle/input/banknote-detection-authentication/data_banknote_authentication.txt",
                        names = ["Variance", "Skewness", "Curtosis", "Entropy", "TargetClass"])

separate dataset into features and target

In [3]:
# separate independent variables
x_orig = banknotes.iloc[:, 0:-1].values 
  
# get the target class
y_orig = banknotes.iloc[:, -1:].values 

onehotencode our target class. set the training parameters such as learning rate and epochs

In [4]:

oneHot = OneHotEncoder() 
  

oneHot.fit(y_orig) 
y = oneHot.transform(y_orig).toarray() 

x = x_orig

  
alpha, epochs = 0.0035, 1000
m, n = x.shape 

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


must define input graphs, weight, and bias

In [5]:
# Number of independent variables n
X = tf.placeholder(tf.float32, [None, n]) 
  
# For binary classification, we only have 2 possible outcomes
Y = tf.placeholder(tf.float32, [None, 2]) 
  
# Model weights 
W = tf.Variable(tf.zeros([n, 2])) 
  
# Model bias
b = tf.Variable(tf.zeros([2])) 

some more setup functions are required such as the hypothesis, cost function, optimizer.

In [ ]:
# Hypothesis 
Y_hat = tf.nn.sigmoid(tf.add(tf.matmul(X, W), b)) 
  
# Sigmoid Cross Entropy Cost Function 

cost = tf.nn.sigmoid_cross_entropy_with_logits( 
                    logits = Y_hat, labels = Y) 
  
# Gradient Descent Optimizer 
optimizer = tf.train.GradientDescentOptimizer( 
         learning_rate = alpha).minimize(cost) 
  
# Global Variables Initializer 
init = tf.global_variables_initializer()

the training session begins

In [8]:

with tf.Session() as sess: 

    sess.run(init) 
      
    # Lists for storing the changing Cost and Accuracy in every Epoch 
    cost_history, accuracy_history = [], [] 
      
    # Iterating through all the epochs 
    for epoch in range(epochs): 
        cost_per_epoch = 0
          
        # Running the Optimizer 
        sess.run(optimizer, feed_dict = {X : x, Y : y}) 
          
        # Calculating cost on current Epoch 
        c = sess.run(cost, feed_dict = {X : x, Y : y}) 
        
        # Calculating accuracy on current Epoch 
        correct_prediction = tf.equal(tf.argmax(Y_hat, 1), 
                                          tf.argmax(Y, 1)) 
        
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, 
                                                 tf.float32)) 
          
        # Storing Cost and Accuracy to the history 
        cost_history.append(sum(sum(c))) 
        accuracy_history.append(accuracy.eval({X : x, Y : y}) * 100) 
          
        # Displaying result on current Epoch 
        if epoch % 100 == 0 and epoch != 0: 
            print("Epoch " + str(epoch) + " Cost: "
                            + str(cost_history[-1])) 
      
    Weight = sess.run(W) # Optimized Weight
    Bias = sess.run(b)   # Optimized Bias
      
    # Final Accuracy 
    correct_prediction = tf.equal(tf.argmax(Y_hat, 1), 
                                      tf.argmax(Y, 1)) 
    
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,  
                                             tf.float32)) 
    print("\nAccuracy:", accuracy_history[-1], "%") 

NameError: name 'init' is not defined

plot the cost per epoch

In [ ]:
plt.plot(list(range(epochs)), cost_history) 
plt.xlabel('Epochs') 
plt.ylabel('Cost') 
plt.title('Cost Per Epoch') 

plt.show() 

plot the accuracy per epoch

In [ ]:
plt.plot(list(range(epochs)), accuracy_history) 
plt.xlabel('Epochs') 
plt.ylabel('Accuracy') 
plt.title('Accuracy Per Epoch') 
  
plt.show() 